# **GA4 Bidder**

In [ ]:
###########################################################################
#
#  Copyright 2021 Google Inc.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      https://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
# This solution, including any related sample code or data, is made available
# on an “as is,” “as available,” and “with all faults” basis, solely for
# illustrative purposes, and without warranty or representation of any kind.
# This solution is experimental, unsupported and provided solely for your
# convenience. Your use of it is subject to your agreements with Google, as
# applicable, and may constitute a beta feature as defined under those
# agreements.  To the extent that you make any data available to Google in
# connection with your use of the solution, you represent and warrant that you
# have all necessary and appropriate rights, consents and permissions to permit
# Google to use and process that data.  By using any portion of this solution,
# you acknowledge, assume and accept all risks, known and unknown, associated
# with its usage, including with respect to your deployment of any portion of
# this solution in your systems, or usage in connection with your business,
# if at all.
###########################################################################

# 1) Define Feature Set

### 1.1) Imports, Globals, & Helper Functions

In [ ]:
#@title
# Load libraries and authenticate colab for BQ
!pip install --upgrade -q gspread | grep -v 'already satisfied'
!pip install gspread_formatting | grep -v 'already satisfied'

import gspread
import math
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.auth import default
from gspread_formatting import *
from multiprocessing import Process
import ipywidgets as widgets
auth.authenticate_user()
bigquery.USE_LEGACY_SQL = False

dc = {}

def query_to_df(query, project_id, dialect='standard'):
  """Returns a dataframe w/ the result of query.
  """

  # Run the query and save to a dataframe
  df = pd.read_gbq(query, project_id=project_id, dialect=dialect)

  return df

def build_eda_query(label_query, ga_project_id, ga_dataset, ga_data, filters):
  return f"""
    WITH

    {label_query},

    event_list as (
      select distinct user_pseudo_id, event_name,
      CASE
      WHEN event_name in ('ad_click','ad_exposure','ad_impression','ad_query','ad_reward','adunit_exposure','app_clear_data','app_remove','app_store_refund','app_store_subscription_cancel','app_store_subscription_convert','app_store_subscription_renew','app_update','dynamic_link_app_open','dynamic_link_app_update','error','firebase_campaign','firebase_in_app_message_dismiss','firebase_in_app_message_impression','first_open','first_visit','in_app_purchase','notification_dismiss','notification_foreground','notification_open','notification_receive','notification_send','os_update','screen_view','session_start','user_engagement','click','file_download','first_visit','page_view','scroll','session_start','user_engagement','video_complete','video_progress','video_start','view_search_results')
        THEN 'standard'
      WHEN event_name in ('earn_virtual_currency','join_group','login','purchase','refund','search','select_content','share','sign_up','spend_virtual_currency','tutorial_begin','tutorial_complete','add_payment_info','add_shipping_info','add_to_cart','add_to_wishlist','begin_checkout','generate_lead','remove_from_cart','select_item','select_promotion','view_cart','view_item_list','view_promotion','level_end','level_start','level_up','post_score','unlock_achievement')
      THEN 'recommended'
      ELSE 'custom' END
      AS event_type
      from `{ga_project_id}.{ga_dataset}.{ga_data}`
      where {filters}
      )

    select event_name, event_type, count(distinct user_pseudo_id) as total_users, count(distinct case when label=1 then user_pseudo_id else null end) as total_converters,
    count(distinct case when label=1 then user_pseudo_id else null end)/count(distinct user_pseudo_id) as  share_converters,
    from
    (select a.*, b.label from event_list a join visitors_labeled b on a.user_pseudo_id=b.user_pseudo_id)
    group by 1,2
    order by 5 desc"""

def create_build_feature_set_sql(label_query, ga_project_id, ga_dataset, ga_data, filters, end_date, event_level_query):
  return f"""
    WITH

    {label_query},


  visitor_city AS (
      SELECT
        user_pseudo_id, metro AS dma
      FROM (
        SELECT c.user_pseudo_id,c.metro, ROW_NUMBER() OVER (PARTITION BY c.user_pseudo_id ORDER BY visits DESC) AS row_num
        FROM (
          SELECT a.user_pseudo_id, geo.metro AS metro, COUNT(*) AS visits
          FROM `{ga_project_id}.{ga_dataset}.{ga_data}` a
          left join visitors_labeled b
          on a.user_pseudo_id = b.user_pseudo_id
          where {filters}
          and (a.event_timestamp < IFNULL(first_transaction_session, 0)
          or first_transaction_session is null)
          GROUP BY user_pseudo_id,metro ) c )
    WHERE row_num = 1 ),

  visitor_common_daypart AS (
    SELECT user_pseudo_id, daypart
    FROM (
      SELECT user_pseudo_id, daypart, ROW_NUMBER() OVER (PARTITION BY user_pseudo_id ORDER BY pageviews DESC) AS row_num
      FROM (
        SELECT
          user_pseudo_id,
          CASE WHEN hour_of_day_localized >= 1 AND hour_of_day_localized < 6 THEN 'night_1_6'
          WHEN hour_of_day_localized >= 6 AND hour_of_day_localized < 11 THEN 'morning_6_11'
          WHEN hour_of_day_localized >= 11 AND hour_of_day_localized < 14 THEN 'lunch_11_14'
          WHEN hour_of_day_localized >= 14 AND hour_of_day_localized < 17 THEN 'afternoon_14_17'
          WHEN hour_of_day_localized >= 17 AND hour_of_day_localized < 19 THEN 'dinner_17_19'
          WHEN hour_of_day_localized >= 19 AND hour_of_day_localized < 22 THEN 'evening_19_23'
          WHEN hour_of_day_localized >= 22 OR hour_of_day_localized = 0 THEN 'latenight_23_1'
          END AS daypart,
          SUM(pageviews) AS pageviews
        FROM (
          SELECT a.user_pseudo_id, EXTRACT(HOUR
            FROM (
                CASE WHEN c.dst = 1 AND event_date BETWEEN '20170312' AND '20171105' THEN TIMESTAMP_ADD(TIMESTAMP_MICROS(event_timestamp), INTERVAL c.timezone+1 HOUR)
                WHEN c.dst = 1 AND event_date BETWEEN '20180311' AND '20181104' THEN TIMESTAMP_ADD(TIMESTAMP_MICROS(event_timestamp), INTERVAL c.timezone+1 HOUR)
                WHEN c.dst = 1 AND event_date BETWEEN '20190310' AND '20191103' THEN TIMESTAMP_ADD(TIMESTAMP_MICROS(event_timestamp), INTERVAL c.timezone+1 HOUR)
                ELSE TIMESTAMP_ADD(TIMESTAMP_MICROS(event_timestamp), INTERVAL c.timezone HOUR)
                END )) AS hour_of_day_localized,
                case when event_name = 'page_view' then 1 else 0 end as pageviews
          FROM `{ga_project_id}.{ga_dataset}.{ga_data}` a
          LEFT JOIN ( SELECT state_name, MAX(CASE WHEN state_name = 'Oregon' THEN -8 ELSE timezone END) AS timezone, MAX(dst) AS dst
            FROM (SELECT states.*
                      FROM UNNEST ([STRUCT("Alaska" as state_name, -9 as timezone, 1 as dst),
                        STRUCT("American Samoa" as state_name, -10 as timezone, 0 as dst),
                        STRUCT("Hawaii" as state_name, -10 as timezone, 0 as dst),
                        STRUCT("California" as state_name, -7 as timezone, 1 as dst),
                        STRUCT("Idaho" as state_name, -7 as timezone, 1 as dst),
                        STRUCT("Nevada" as state_name, -8 as timezone, 1 as dst),
                        STRUCT("Oregon" as state_name, -8 as timezone, 1 as dst),
                        STRUCT("Washington" as state_name, -8 as timezone, 1 as dst),
                        STRUCT("Arizona" as state_name, -7 as timezone, 0 as dst),
                        STRUCT("Colorado" as state_name, -7 as timezone, 1 as dst),
                        STRUCT("Kansas" as state_name, -6 as timezone, 1 as dst),
                        STRUCT("Montana" as state_name, -7 as timezone, 1 as dst),
                        STRUCT("North Dakota" as state_name, -6 as timezone, 1 as dst),
                        STRUCT("Nebraska" as state_name, -6 as timezone, 1 as dst),
                        STRUCT("New Mexico" as state_name, -7 as timezone, 1 as dst),
                        STRUCT("South Dakota" as state_name, -6 as timezone, 1 as dst),
                        STRUCT("Texas" as state_name, -6 as timezone, 1 as dst),
                        STRUCT("Utah" as state_name, -7 as timezone, 1 as dst),
                        STRUCT("Wyoming" as state_name, -7 as timezone, 1 as dst),
                        STRUCT("Alabama" as state_name, -6 as timezone, 1 as dst),
                        STRUCT("Arkansas" as state_name, -6 as timezone, 1 as dst),
                        STRUCT("Florida" as state_name, -5 as timezone, 1 as dst),
                        STRUCT("Iowa" as state_name, -6 as timezone, 1 as dst),
                        STRUCT("Illinois" as state_name, -6 as timezone, 1 as dst),
                        STRUCT("Indiana" as state_name, -5 as timezone, 1 as dst),
                        STRUCT("Kentucky" as state_name, -5 as timezone, 1 as dst),
                        STRUCT("Louisiana" as state_name, -6 as timezone, 1 as dst),
                        STRUCT("Michigan" as state_name, -5 as timezone, 1 as dst),
                        STRUCT("Minnesota" as state_name, -6 as timezone, 1 as dst),
                        STRUCT("Missouri" as state_name, -6 as timezone, 1 as dst),
                        STRUCT("Mississippi" as state_name, -6 as timezone, 1 as dst),
                        STRUCT("Oklahoma" as state_name, -6 as timezone, 1 as dst),
                        STRUCT("Tennessee" as state_name, -5 as timezone, 1 as dst),
                        STRUCT("Wisconsin" as state_name, -6 as timezone, 1 as dst),
                        STRUCT("Connecticut" as state_name, -5 as timezone, 1 as dst),
                        STRUCT("District of Columbia" as state_name, -5 as timezone, 1 as dst),
                        STRUCT("Delaware" as state_name, -5 as timezone, 1 as dst),
                        STRUCT("Georgia" as state_name, -5 as timezone, 1 as dst),
                        STRUCT("Massachusetts" as state_name, -5 as timezone, 1 as dst),
                        STRUCT("Maryland" as state_name, -5 as timezone, 1 as dst),
                        STRUCT("Maine" as state_name, -5 as timezone, 1 as dst),
                        STRUCT("North Carolina" as state_name, -5 as timezone, 1 as dst),
                        STRUCT("New Hampshire" as state_name, -5 as timezone, 1 as dst),
                        STRUCT("New Jersey" as state_name, -5 as timezone, 1 as dst),
                        STRUCT("New York" as state_name, -5 as timezone, 1 as dst),
                        STRUCT("Ohio" as state_name, -5 as timezone, 1 as dst),
                        STRUCT("Pennsylvania" as state_name, -5 as timezone, 1 as dst),
                        STRUCT("Rhode Island" as state_name, -5 as timezone, 1 as dst),
                        STRUCT("South Carolina" as state_name, -5 as timezone, 1 as dst),
                        STRUCT("Virginia" as state_name, -5 as timezone, 1 as dst),
                        STRUCT("Vermont" as state_name, -5 as timezone, 1 as dst),
                        STRUCT("West Virginia" as state_name, -5 as timezone, 1 as dst),
                        STRUCT("Puerto Rico" as state_name, -4 as timezone, 0 as dst),
                        STRUCT("Virgin Islands" as state_name, -4 as timezone, 0 as dst)]) states)
            GROUP BY state_name) c
          ON a.geo.region = c.state_name
          left join visitors_labeled b
          on a.user_pseudo_id = b.user_pseudo_id
          where {filters}
          and (a.event_timestamp < IFNULL(first_transaction_session, 0)
          or first_transaction_session is null)
          )
        GROUP BY 1, 2 ) )
    WHERE row_num = 1 ),

  visitor_common_day AS (
    SELECT user_pseudo_id, case when day = 1 then "Sunday"
    when day = 2 then "Monday"
    when day = 3 then "Tuesday"
    when day = 4 then "Wednesday"
    when day = 5 then "Thursday"
    when day = 6 then "Friday"
    when day = 7 then "Saturday" end as day
    FROM (
      SELECT user_pseudo_id, day, ROW_NUMBER() OVER (PARTITION BY user_pseudo_id ORDER BY pages_viewed DESC) AS row_num
      FROM (
        SELECT a.user_pseudo_id, EXTRACT(DAYOFWEEK FROM PARSE_DATE('%Y%m%d',event_date)) AS day, sum(case when event_name = 'page_view' then 1 else 0 end) AS pages_viewed
        FROM `{ga_project_id}.{ga_dataset}.{ga_data}` a
        left join visitors_labeled b
        on a.user_pseudo_id = b.user_pseudo_id
        where {filters}
        and (a.event_timestamp < IFNULL(first_transaction_session, 0)
        or first_transaction_session is null)
        GROUP BY user_pseudo_id, day ) )
    WHERE row_num = 1 ),


  engagement as (
    select
        user_pseudo_id,
        safe_divide(sum(session_engaged),count(distinct session_id))  as engagement_rate,
        round(sum(engagement_time_msec)/1000) as engagement_time_seconds,
        safe_divide(sum(distinct case when session_engaged = 0 then 1 else 0 end),count(distinct session_id)) as bounce_rate,
        count(distinct session_id) as total_sessions
    from (
        select
            user_pseudo_id,
            (select value.int_value from unnest(event_params) where key = 'ga_session_id') as session_id,
            max((select value.int_value from unnest(event_params) where key = 'session_engaged')) as session_engaged,
            max((select value.int_value from unnest(event_params) where key = 'engagement_time_msec')) as engagement_time_msec
    from `{ga_project_id}.{ga_dataset}.{ga_data}` a
    where {filters}
        group by
            user_pseudo_id,
            session_id)
    group by user_pseudo_id
    ),

  users_sessions as (
  select z.*, b.dma as visited_dma,
    c.daypart as visited_daypart,
    d.day as visited_dow,
    e.engagement_rate,
    e.engagement_time_seconds,
    e.total_sessions
    from (
    SELECT a.user_pseudo_id,
    max(label) as label,
    sum(case when event_name = 'page_view' then 1 else 0 end) as pageviews,
    safe_divide(sum(case when event_name = 'page_view' then 1 else 0 end), count(distinct a.user_pseudo_id)) as avg_session_depth,
    max(case when device.category = 'mobile' then 1 else 0 end) as mobile,
    max(case when device.web_info.browser = 'Chrome' then 1 else 0 end) as Chrome,
    max(case when device.web_info.browser = 'Safari' then 1 else 0 end) as Safari,
    max(case when device.web_info.browser <> 'Chrome' and device.web_info.browser not like '%Safari%' then 1 else 0 end) as browser_other,
    sum(case when traffic_source.medium = '(none)' then 1 else 0 end) as visits_traffic_source_none,
    sum(case when traffic_source.medium = 'organic' then 1 else 0 end) as visits_traffic_source_organic,
    sum(case when traffic_source.medium = 'cpc' then 1 else 0 end) as visits_traffic_source_cpc,
    sum(case when traffic_source.medium = 'cpm' then 1 else 0 end) as visits_traffic_source_cpm,
    sum(case when traffic_source.medium = 'affiliate' then 1 else 0 end) as visits_traffic_source_affiliate,
    sum(case when traffic_source.medium = 'referral' then 1 else 0 end) as visits_traffic_source_referral,
    count(distinct geo.metro) as distinct_dmas,
    count(distinct EXTRACT(DAYOFWEEK FROM PARSE_DATE('%Y%m%d', event_date))) as num_diff_days_visited,
    case when min(first_transaction_date) is null then DATE_DIFF(PARSE_DATE('%Y%m%d', CAST({end_date} as STRING)), min(PARSE_DATE('%Y%m%d',event_date)), DAY) else DATE_DIFF(min(PARSE_DATE('%Y%m%d',first_transaction_date)), min(PARSE_DATE('%Y%m%d',event_date)), DAY) end as days_since_first_visit,
    case when min(first_transaction_date) is null then DATE_DIFF(PARSE_DATE('%Y%m%d', CAST({end_date} as STRING)), MAX(PARSE_DATE('%Y%m%d',event_date)), DAY) else DATE_DIFF(min(PARSE_DATE('%Y%m%d',first_transaction_date)), MAX(PARSE_DATE('%Y%m%d',event_date)), DAY) end as days_since_last_visit
    from `{ga_project_id}.{ga_dataset}.{ga_data}` a
    left join visitors_labeled b
    on a.user_pseudo_id = b.user_pseudo_id
    where {filters}
    and (a.event_timestamp < IFNULL(first_transaction_session, 0)
    or first_transaction_session is null)
    group by a.user_pseudo_id) z
    left join visitor_city b
    on z.user_pseudo_id = b.user_pseudo_id
    left join visitor_common_daypart c
    on z.user_pseudo_id = c.user_pseudo_id
    left join visitor_common_day d
    on z.user_pseudo_id = d.user_pseudo_id
    left join engagement e
    on z.user_pseudo_id = e.user_pseudo_id)

  select a.*, b.* except (user_pseudo_id), case when RAND() < 0.10 then 1 else 0 end as hold_out
  from users_sessions a
  left join (
    {event_level_query}
    left join visitors_labeled b
    on a.user_pseudo_id = b.user_pseudo_id
    where {filters}
    and (a.event_timestamp < IFNULL(first_transaction_session, 0)
    or first_transaction_session is null)
    group by 1
  ) b
  on a.user_pseudo_id = b.user_pseudo_id
  where a.engagement_rate > 0;
  """

  # Define functions to get correlation matrix and plot distribution

def corr_matrix(df, absolute=False, force_dtype=True, figsize=(20, 16)):
  """Plot a heatmap of a correlation matrix.

  Keyword arguments:
    df -- a dataframe
    absolute -- Takes absolute values of correlation coefficient (default to False)
    force_dtype -- Tries to transform all object dtypes into floats (default True)
  """

  # Make a copy of the dataframe
  df2 = df.copy()

  # Transform data types to floats to prep for correlation matrix, since objects cannot be an input
  if force_dtype:
    for col, dtype in zip(df2.columns.values, df2.dtypes):
      if dtype == 'object':
        try:
          df2[col] = df2[col].astype(float)
        except ValueError:
          try:
            df2[col] = df2[col].str.rstrip('%').astype('float') / 100.0
          except ValueError:
            print(f"Error transforming {col} into float! Removing from correlation matrix.")
            df2 = df2.drop(columns=col)


  # Generate correlation matrix
  if absolute:
    corr = df2.corr().abs()
  else:
    corr = df2.corr()

  # Generate a mask for the upper triangle
  mask = np.zeros_like(corr, dtype=np.bool)
  mask[np.triu_indices_from(mask)] = True

  # Set up the matplotlib figure
  f, ax = plt.subplots(figsize=figsize)

  # Generate a custom diverging colormap
  cmap = sns.diverging_palette(10, 140, as_cmap=True)

  # Draw the heatmap with the mask and correct aspect ratio
  if absolute:
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, vmin=0, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})
  else:
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, vmin=-1, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})

  plt.title("Correlation Matrix", loc='center', fontsize='large', fontweight='bold')

  return corr

def plot_density(df, cols=None, scaled='normalize', title="Density Plot", figsize=(8, 5),
                 hist=True, xlabel="Distribution", ylabel="Density"):
  """Plots density of multiple variables and labels them.

  Keyword arguments:
    df -- input DataFrame
    cols -- list of column names to plot (default None, will plot all data if possible)
    scaled -- method of scaling data (default "normalize", can be set to None)
    title -- title of chart
    figsize -- pyplot figsize
    hist -- Boolean, include histogram or not (default False)
  """
  def _scale_data(df, cols=None, method="normalize"):
    # Make a copy of the dataframe
    df_copy = df.copy(deep=True)

    # loop through columns to normalize
    if cols is None:
      cols = df.columns.values

    for col in cols:
      if method == "normalize":
        try:
          df_copy[col] = (df_copy[col]-df_copy[col].min())/(df_copy[col].max()-df_copy[col].min())
        except:
          None
      if method == "standardize":
        try:
          df_copy[col] = (df_copy[col]-df_copy[col].mean())/df_copy[col].std()
        except:
          None

    return df_copy
  # make a copy of dataframe
  df_copy = df.copy(deep=True)

  # Check dtypes for non-objects, transform as floats if possible
  for field, dtype in zip(df_copy.columns.values, df_copy.dtypes):
    if dtype == 'object':
      try:
        df_copy[field] = df_copy[field].astype(float)
      except ValueError:
        print(f"Error transforming {field} into float! Removing from density plot.")
        df_copy = df_copy.drop(columns=field)

  # Scale data based on input
  if scaled == 'normalize':
    df_copy = _scale_data(df_copy, cols=cols, method='normalize')
  elif scaled == 'standardize':
    df_copy = _scale_data(df_copy, cols=cols, method='standardize')
  else:
    xlabel = "Values"

  # set figsize on figure object
  plt.figure(figsize=figsize)

  # Plot each field
  if cols is None:
    for field in df_copy.columns.values:
      sns.distplot(df_copy[field], hist=hist, label=field,
                   kde=True, kde_kws = {'linewidth': 2})
  else:
    for field in cols:
      sns.distplot(df_copy[field], hist=hist, label=field,
                   kde=True, kde_kws = {'linewidth': 2})


  plt.legend(title="Legend")
  plt.title(title, loc='center', fontsize='large', fontweight='bold')
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)
  #plt.show()

def compare_labels(df, col, label='label', user_col='num_users', figsize=(9,5),
                   trim_max=None, trim_min=None):
  """
  Sums 'user_col' by inputs 'label' and 'col', then plots the two labels against each other.
  Defaults to using 'label' as the label column and 'num_users' as the count of users column.

  Keyword arguments:
    df -- input dataframe
    col -- column in df you want to compare

  Arguments:
    label = 'label' -- name of label column; label of audiences you want to compare
    user_col = 'num_users' -- column which should contain the count of users in your dataset
    figsize = (9,5) -- figsize of resulting figures to plot
  """

  df2 = df.copy(deep=True)
  df2 = pd.DataFrame(df2.groupby([label, col])[user_col].sum())
  df2.reset_index(inplace=True)

  if trim_max is not None:
    df2[col] = df2[col].astype(float)
    df2 = df2.loc[df2[col] <= float(trim_max)]

  if trim_min is not None:
    df2[col] = df2[col].astype(float)
    df2 = df2.loc[df2[col] >= float(trim_max)]

  fig, ax1 = plt.subplots(figsize=figsize)
  ax2 = ax1.twinx()

  ln1 = ax1.plot(np.array(df2.loc[df2[label] == 1, col]), np.array(df2.loc[df2[label] == 1, user_col]),
    color='r', label='1')

  ln2 = ax2.plot(np.array(df2.loc[df2[label] == 0, col]), np.array(df2.loc[df2[label] == 0, user_col]),
    color='b', label='0')

  lns = ln1+ln2
  labs = [l.get_label() for l in lns]
  ax1.legend(lns, labs, loc=0)

  plt.title(col)

# Helper function to look at distributions of data
def query_and_compare(col, trim_max=None, trim_min=None, figsize=None):
  dc['col'] = col
  query = f"""
  select label, {col}, count(*) as num_users
  from `{destination_project_id}.{destination_dataset}.{destination_table}`
  where engagement_rate > 0
  group by 1, 2
  """

  df = query_to_df(query, destination_project_id)

  compare_labels(df, col=col, figsize=figsize, trim_max=trim_max,
                 trim_min=trim_min)

### 1.2) Set Parameters to Pull in GA4 Export Data from BigQuery

In [ ]:
# Fill in query / feature

ga_project_id = "" #@param {type:"string"}
ga_dataset = "" #@param {type:"string"}
ga_data = "" #@param {type:"string"}
start_date = "" #@param {type:"string"}
end_date = "" #@param {type:"string"}

### 1.3) Filtering GA Export Data Based on Dates Above

*   [Optional]: Add additional filters based on the business use case



In [ ]:
# @title
filters = f"""
  _TABLE_SUFFIX BETWEEN '{start_date}' AND '{end_date}'
  AND geo.Country="United States"
  """


In [ ]:
# @title
#value_query = f"""
#  SELECT event_name, SUM(rev)/COUNT(user_pseudo_id) AS value FROM
#  (SELECT a.event_name, a.user_pseudo_id, AVG(b.rev)/COUNT(a.event_name) AS rev FROM (select * from `{ga_project_id}.{ga_dataset}.{ga_data}` WHERE {filters}) a INNER JOIN
#  (SELECT user_pseudo_id, SUM(ecommerce.purchase_revenue) AS rev FROM `{ga_project_id}.{ga_dataset}.{ga_data}` WHERE {filters} GROUP BY user_pseudo_id) b
#  ON a.user_pseudo_id = b.user_pseudo_id GROUP BY a.event_name, a.user_pseudo_id) GROUP BY event_name
#"""

conversion_value_query = f"""
  select event_name,  COUNT(event_name), SUM(ecommerce.purchase_revenue) from `{ga_project_id}.{ga_dataset}.{ga_data}` WHERE {filters} GROUP BY event_name
"""


### 1.4) Define Target Variable and Use Case

**Instructions:** Run block of code, select "Conversion Action" and "Bid Towards", then move to next block of code. Do no re-run this block.

**Use Case #1**: Online conversions  and revenue available

*   Conversion Action: Select conversion event
*   Bid Towards: Revenue

**Use Case #2**: Non revenue conversion event (i.e. lead form, request additional info, etc.)

*   Conversion Action: Select conversion event
*   Bid Towards: Low Funnel Activity

**Use Case #3**: Deeper business goal (i.e. >2 purchases in 6 months, conversions > $500, etc.)

*   Conversion Action: Select conversion event
*   Bid Towards: Deeper Business Goal




In [ ]:
# @title
conversion_value_mapping = query_to_df(conversion_value_query, ga_project_id)
conversion_value_mapping.set_index('event_name', inplace=True)
conversion_dropdown = widgets.Dropdown(
    options=conversion_value_mapping.index.tolist(),
    description='Conversion Action:',
    disabled=False,
    style={'description_width': 'initial'}
)

bid_towards_dropdown = widgets.Dropdown(
    options=['Revenue', 'Low Funnel Activity', 'Deeper Business Goal'],
    description='Bid Towards:',
    disabled=False,
    style={'description_width': 'initial'}
)
widgets.HBox([conversion_dropdown, bid_towards_dropdown])

In [ ]:
# @title
conversion_event = conversion_dropdown.value
def build_top_k_action_query(actions, is_only_converters):
  actions_query = ''
  for i in actions:
    actions_query += "'" + i + "' in UNNEST(a) and "
  actions_query = actions_query[:-4]
  converter_proportion_top_k = f"""
    select count(distinct user_pseudo_id) from `{ga_project_id}.{ga_dataset}.{ga_data}` WHERE {filters} and user_pseudo_id in
      (select user_pseudo_id from
        (select user_pseudo_id, ARRAY_AGG(event_name) as a FROM `{ga_project_id}.{ga_dataset}.{ga_data}` WHERE {filters} group by user_pseudo_id)
      where
      {actions_query}
    ) {'and event_name = "' + conversion_event + '"' if is_only_converters else ''}
  """
  return converter_proportion_top_k

### 1.5) Initial EDA at the Event Level

Learn more about event type [here](https://support.google.com/analytics/answer/9322688?hl=en#zippy=%2Crealtime-report%2Cdebugview-report).

In [ ]:
# @title
label_query = f"""
  visitors_labeled as (
  select user_pseudo_id,
  min(case when event_name = '{conversion_event}' then event_timestamp end) as first_transaction_session,
  min(case when event_name = '{conversion_event}' then event_date end) as first_transaction_date,
  max(case when event_name = '{conversion_event}' then 1 else 0 end) as label
  from `{ga_project_id}.{ga_dataset}.{ga_data}` a
  where {filters}
  group by user_pseudo_id
  )"""
query_to_df(build_eda_query(label_query, ga_project_id, ga_dataset, ga_data, filters), ga_project_id)

###1.6) Create event level dataset

In [ ]:
# @title
#  [Optional] add additional features manually
#  Adding additional unnests will cause duplicates without additional edits
#  to the code.

#  event_level_query = f"""select a.user_pseudo_id,
#  max(case when event_name = 'add_payment_info' then 1 else 0 end) as campus_collection_user_flag,
#  max(case when event_name = 'begin_checkout' then 1 else 0 end) as view_item_list_flag,
#  max(case when event_name = 'add_to_cart' then 1 else 0 end) as san_francisco_users_flag,
#  max(case when event_name = 'select_item' then 1 else 0 end) as android_lovers_flag,
#  max(case when event_name = 'view_item' then 1 else 0 end) as select_promotion_flag,
#  max(case when event_name = 'select_promotion' then 1 else 0 end) as view_promotion_flag

#  from `{ga_project_id}.{ga_dataset}.{ga_data}` a"""

event_level_query = f"""select a.user_pseudo_id,
"""
for event in conversion_value_mapping.index.tolist():
  if event != '404_error' and event != conversion_event:
    event_col_name = re.sub(r'[ -]', '_', event)
    event_col_name = re.sub(r'[^a-zA-Z0-9_]', '', event_col_name)
    event_level_query += f"""
      max(case when event_name = '{event}' then 1 else 0 end) as {event_col_name},"""
event_level_query += f"""
  from `{ga_project_id}.{ga_dataset}.{ga_data}` a"""

###1.7) Save final dataset to BigQuery
Dataset and Table names must only include letters, numbers, and underscores


In [ ]:
destination_project_id = "" #@param {type:"string"}
destination_dataset = "" #@param {type:"string"}
destination_table = "" #@param {type:"string"}

client = bigquery.Client(destination_project_id)

try:
  dataset = bigquery.Dataset(destination_project_id + '.' +destination_dataset)
  dataset.location = "US"
  dataset = client.create_dataset(dataset, timeout=30)
  print("Created dataset {}.{}".format(client.project, dataset.dataset_id))
except:
  print('Dataset already exists, skipping creation...')


job_config = bigquery.QueryJobConfig()
table_ref = client.dataset(destination_dataset).table(destination_table)
job_config.destination = table_ref
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

# Start the query, passing in the extra configuration.
query_job = client.query(
    create_build_feature_set_sql(label_query, ga_project_id, ga_dataset, ga_data, filters, end_date, event_level_query),
    location="US",
    job_config=job_config
)

query_job.result()
print(f"Query results loaded to table {table_ref.path}")

# 2) Exploratory Data Analysis (EDA)


### 2.1)  Check Distribution of Labels

Conversion data is often **imbalanced** (significantly more non converters than converters). We will use the ratio to help balance the dataset for model training.

For more info click [here](https://developers.google.com/machine-learning/data-prep/construct/sampling-splitting/imbalanced-data).

In [ ]:
# @title
query = f"select label, count(*) as users from `{destination_project_id}.{destination_dataset}.{destination_table}` group by 1;"
dist = query_to_df(query, destination_project_id)
print(dist)
print("\nRatio of 0's to 1's: ")
dist.set_index('label', inplace=True)
print(dist.at[0,'users']/dist.at[1,'users'])
sample_rate = int(dist.at[0,'users']/dist.at[1,'users'])

### 2.2) Correlation Matrix

In [ ]:
# @title
query = f"select label, count(*) as users from `{destination_project_id}.{destination_dataset}.{destination_table}` group by 1;"
users_to_sample = 50000
sampler = float(1/(dist.sum()/users_to_sample))
dist = query_to_df(query, destination_project_id)
dist.set_index('label', inplace=True)
dist.at[0,'users']/dist.at[1,'users']
sampled_data = query_to_df(f'select * from `{destination_project_id}.{destination_dataset}.{destination_table}` where RAND() < {sampler}', destination_project_id)
corr = corr_matrix(sampled_data, figsize=(12,9))

### 2.3) [Optional]: Enter your email address to receive correlation matrix via Google Sheets

In [ ]:
#@title
email = "" #@param {type:"string"}

creds, _ = default()
gc = gspread.authorize(creds)
#sh = gc.open_by_url(sheet_url)
#corr = corr_matrix(sampled_data, figsize=(12,9))

sh = gc.create('Correlation Matrix')
worksheet = sh.add_worksheet(title="Corr", rows="1000", cols="100")
sh.del_worksheet(sh.sheet1)

def num_to_col_letters(num):
    letters = ''
    while num:
        mod = (num-1) % 26
        letters += chr(mod + 65)
        num = (num-1) // 26
    return ''.join(reversed(letters))

def df_to_sheet(df, sh):

  ran = 'A1:'+num_to_col_letters(df.shape[1])+str(df.shape[0]+1)
  cell_list = worksheet.range(ran)
  df_list = df.values.tolist()
  flat_list = [item for sublist in df_list for item in sublist]
  list_headers = list(df.columns.values)
  flat_list=list_headers+flat_list

  for cell, value in zip(cell_list, flat_list):
    if value == value:
      cell.value = value
    else:
      cell.value = 0
  worksheet.update_cells(cell_list)

rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('A1:'+num_to_col_letters(corr.shape[1])+str(corr.shape[0]+1), worksheet)],
    gradientRule=GradientRule(
        minpoint=InterpolationPoint(color=Color(204/255,0,0), value = '-1', type='NUMBER'),
        midpoint=InterpolationPoint(color=Color(1,1,1), value = '0', type='NUMBER'),
        maxpoint=InterpolationPoint(color=Color(56/255,118/255,29/255), value = '1', type='NUMBER')
    )
)

rules = get_conditional_format_rules(worksheet)
rules.append(rule)
rules.save()

df_to_sheet(corr, worksheet)

worksheet.insert_cols([[None] + list(corr.columns.values)])

sh.share(email, perm_type='user', role='writer')
print("Correlation matrix has been shared to: {}".format(email))


# 3) Model Training & Evaluation

### 3.1) Remove Fields Not Needed in Final Model

*   Seperate fields with a comma
* Remove  "user_pseudo_id"
* Remove any variable that is perfectly correlated with your target variable (i.e. target var is purchase, you would want to remove a "Thank you" page visit)



In [ ]:
remove_fields = "hold_out, user_pseudo_id, days_since_last_visit, days_since_first_visit" #@param {type:"string"}

### 3.2) Create Many Iterations of Models

Models tested:

*   Logistic Regression
*   XG Boost

L1 & L2 regularization tested: 0, 0.0001, 0.001, 0.01, 0.1, 0.2, 0.3

FYI: for more information about creating a model in BigQuery, click [here](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create).




In [ ]:
test_mode = True #@param {type:"boolean"}

In [ ]:
#@title
# Start with empty dict and empy list of model names
# By default, only includes logit regression and non-reduced feature list
import math
model_dict = {}
model_name_list = []
model_type_list = ['LOGISTIC_REG', 'BOOSTED_TREE_CLASSIFIER']
Enable_Global_Explainability = True
model_name_base = "ga4_propensity_model_"
model_project = destination_project_id
model_dataset = destination_dataset
label_1_weight = 1.0
label_0_weight = round(math.sqrt(sample_rate/2.0))
l1_reg = ['0', '0.0001', '0.001', '0.01', '0.1', '0.2', '0.3'] if not test_mode else ['0']
l2_reg = ['0', '0.0001', '0.001', '0.01', '0.1', '0.2', '0.3'] if not test_mode else ['0']

standard_features = f"""* except ({remove_fields})"""

for model in client.list_models(destination_dataset):
  client.delete_model(f'{destination_project_id}.{destination_dataset}.{model.model_id}')

def create_model(model, l1_reg_input, l2_reg_input):
  global label_1_weight
  global label_0_weight
  global l1_reg
  global l2_reg
  global model_project
  global model_dataset
  global model_name_base
  global sample_rate
  global Include_XGBoost
  global Enable_Global_Explainability
  global remove_fields

  CLASS_WEIGHTS = f""",CLASS_WEIGHTS = [('1', {label_1_weight}), ('0', {label_0_weight})]"""
  OPTIMIZE_STRATEGY = ''

  model_type = model
  model_prefix = model[0:5]


  model_name = model_name_base + '_' + model_prefix + '_l1_' + str(l1_reg_input).replace(".","_") + '_l2_' + str(l2_reg_input).replace(".","_")
  model_name_list.append(model_name)
  model_dict[model_name] = {}
  model_dict[model_name]['model_type'] = model_type
  client = bigquery.Client(destination_project_id)

  job_config = bigquery.QueryJobConfig()

  model_query = f"""CREATE MODEL
    `{model_project}.{model_dataset}.{model_name}` OPTIONS
    (model_type='{model_type}',
    labels = ['label'],
    L1_REG = {l1_reg_input},
    L2_REG = {l2_reg_input},
    DATA_SPLIT_METHOD = 'RANDOM',
    DATA_SPLIT_EVAL_FRACTION = 0.20
    {CLASS_WEIGHTS}
    {OPTIMIZE_STRATEGY},
    ENABLE_GLOBAL_EXPLAIN = {Enable_Global_Explainability}) AS
    (select * except ({remove_fields}) from `{destination_project_id}.{destination_dataset}.{destination_table}`
    where label = 1 and hold_out = 0
    union all
    select * except ({remove_fields}) from `{destination_project_id}.{destination_dataset}.{destination_table}`
    where label = 0 and hold_out = 0
    and mod(abs(Farm_Fingerprint (user_pseudo_id)),{round(math.sqrt(sample_rate/2))})=0)"""

  query_job = client.query(
    model_query,
    location="US",
    job_config=job_config,
  )

  query_job.result()
  #print("Model training completed")

# Iterate through the model types, and different regularization levels
progress_bar_started = widgets.IntProgress(
    value=0,
    min=0,
    max=98,
    description='Model Training Started:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'white', 'description_width': 'initial'},
    orientation='horizontal'
)
display(progress_bar_started)
procs = []
for model in model_type_list:
  for reg1 in l1_reg:
    for reg2 in l2_reg:
      proc = Process(target=create_model, args=(model, reg1, reg2))
      procs.append(proc)
      proc.start()
      progress_bar_started.value += 1

progress_bar_completed = widgets.IntProgress(
    value=0,
    min=0,
    max=98,
    description='Model Training Finished:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'green', 'description_width': 'initial'},
    orientation='horizontal'
)
display(progress_bar_completed)

for proc in procs:
  proc.join()
  progress_bar_completed.value += 1
print("Model Types Tested: " + str(model_type_list))
print("L1 Reg Tested: " + str(l1_reg))
print("L2 Reg Tested: " + str(l2_reg))
print("All training completed.")

### 3.3) Select Model Performance Metric

**Instructions:** Run block of code, select "Evaluation Metric", then move to next block of code. Do no re-run this block.

This criteria will be used to identify the best fit model.


*   F1 & ROC AUC are best suited for imbalanced data

**Note:** **Before proceeding,** if you would like to view additional details about each model created, like full evaluation metrics & feature importance, you can check them out individually in the BigQuery Console.




In [ ]:
#@title
evaluation_metric = widgets.Dropdown(
    options=[('Accuracy', 'accuracy'), ('Precision', 'precision'), ('Recall', 'recall'), ('F1 Score', 'f1Score'), ('ROC AUC', 'rocAuc')],
    value='accuracy',
    description='Evaluation Metric:',
    disabled=False,
    style={'description_width': 'initial'}
)
evaluation_metric

### 3.4) Select Best Model

We identify the model with the highest performance on the **test data**, according to the criteria selected above.

In [ ]:
# @title
max = 0
best_model = ''
for model in client.list_models(destination_dataset):
  resp = client.get_model(f'{destination_project_id}.{destination_dataset}.{model.model_id}')
  #pp.pprint(resp.__dict__)
  if 'binaryClassificationMetrics' in resp._properties['trainingRuns'][0]['evaluationMetrics'].keys():
    if resp._properties['trainingRuns'][0]['evaluationMetrics']['binaryClassificationMetrics']['aggregateClassificationMetrics'][evaluation_metric.value] > max:
      max = resp._properties['trainingRuns'][0]['evaluationMetrics']['binaryClassificationMetrics']['aggregateClassificationMetrics'][evaluation_metric.value]
      best_model = model.model_id

print(f"Best model: {best_model}")
print(f"{evaluation_metric.value.capitalize()} on holdout sample: {max}")

for model in client.list_models(destination_dataset):
  if best_model != model.model_id :
    client.delete_model(f'{destination_project_id}.{destination_dataset}.{model.model_id}')


### 3.5) Calculate Global Explainability

To learn more about Global Explainability click [here](https://cloud.google.com/bigquery/docs/xai-overview).

In [ ]:
#@title
query=f"""
SELECT
  *
FROM
  ML.GLOBAL_EXPLAIN(MODEL `{model_project}.{model_dataset}.{best_model}`)
"""
#Remove limit statement if all data is needed for deeper analysis
explain=pd.read_gbq(query, project_id=model_project, dialect='standard')
explain

# 4) Generate Custom Bidding Algorithm

### 4.1) Remove Fields Not Needed in Final Script

For instance:
*   Fields that are not biddable
*   Fields with low attribution

Seperate events with a comma.



In [ ]:
ignore_events = "" #@param {type:"string"}
propertyId=0 #@param {type:"integer"}

### 4.2) Generate Custom Bidding Script!

You can copy and paste this script directly into DV360.

If you'd like to have the custom script pushed directly into DV360 from Colab, please continue to the next section.

In [ ]:
#@title
# This constant is defined because we want the sum of the HVAs to be 0.8
sum_HVA_max_coeff = 1.25
explain['attribution_scaled'] = explain['attribution'] / sum(explain['attribution'])
explain['attribution_scaled_no_value'] = explain['attribution'] / (sum(explain['attribution']) * sum_HVA_max_coeff)

script_end = '''
  ])
'''
biddable_attributes = filter(lambda a : a in conversion_value_mapping.index.tolist() and a != conversion_event and a not in [s.strip() for s in ignore_events.split(',')], explain['feature'])
biddable_attributes = list(biddable_attributes)
converters = query_to_df(build_top_k_action_query(actions = [biddable_attributes[0], biddable_attributes[1], biddable_attributes[2]], is_only_converters = True), ga_project_id)['f0_'][0]
all_actions = query_to_df(build_top_k_action_query(actions = [biddable_attributes[0], biddable_attributes[1], biddable_attributes[2]], is_only_converters = False), ga_project_id)['f0_'][0]

avg_conversion_value = (conversion_value_mapping.loc[conversion_event]['f1_'] / conversion_value_mapping.loc[conversion_event]['f0_']) * (converters / all_actions)

script_contents = 'sum_aggregate(['
no_value_flag = False
attribution_coeff_column = 'attribution_scaled'

if bid_towards_dropdown.value == 'Low Funnel Activity':
  attribution_coeff_column = 'attribution_scaled_no_value'
  attribution_entry = explain.loc[explain['feature'] == conversion_event][attribution_coeff_column]
  a_condition = 'sum_aggregate([([has_ga4_conversions('+str(propertyId)+', "'+conversion_event+'")], 1)])'
elif bid_towards_dropdown.value == 'Deeper Business Goal':
  a_condition = '0'
else:
  a_condition = 'ga4_conversions_total_value('+str(propertyId)+', "'+conversion_event+'")'

for index, row in explain.iterrows():
  if row['feature'] in conversion_value_mapping.index.tolist() and row['feature'] != conversion_event and row['feature'] not in ignore_events.split(', '):
    value = row[attribution_coeff_column]
    if  bid_towards_dropdown.value == 'Revenue':
      value = row[attribution_coeff_column] * avg_conversion_value
    script_contents += f'''
        ([has_ga4_conversions({propertyId}, '{row['feature']}')], {value}),'''
b_condition = script_contents + script_end

conditional_script = f'''
_a = {a_condition}
_b = {b_condition}
if _a > 0:
  return _a
else:
  return _b
'''
print(conditional_script)

with open('/content/script.txt', 'w') as writefile:
    writefile.write(conditional_script)

### 4.3) [Optional]: Upload Script Directly to DV360

Input credentials

In [ ]:
import sys
from oauthlib.oauth2.rfc6749.errors import InvalidGrantError
from urllib import parse
from google_auth_oauthlib.flow import InstalledAppFlow
credentials = None

client_id = "" #@param {type:"string"}
client_secret = "" #@param {type:"string"}
partnerId = 0 #@param {type:"integer"}
advertiserId = 0 #@param {type:"integer"}
SCOPES = ['https://www.googleapis.com/auth/adwords',
           'https://www.googleapis.com/auth/dfareporting',
           'https://www.googleapis.com/auth/dfatrafficking',
           'https://www.googleapis.com/auth/ddmconversions',
           "https://www.googleapis.com/auth/analytics.edit",
           'https://www.googleapis.com/auth/spreadsheets.readonly',
           'https://www.googleapis.com/auth/gmail.send',
           'https://www.googleapis.com/auth/cloud-platform',
           'https://www.googleapis.com/auth/display-video'
           ]

class ClientConfigBuilder(object):
      """Helper class used to build a client config dict used in the OAuth 2.0 flow.
      """
      _DEFAULT_AUTH_URI = 'https://accounts.google.com/o/oauth2/auth'
      _DEFAULT_TOKEN_URI = 'https://accounts.google.com/o/oauth2/token'
      CLIENT_TYPE_WEB = 'web'
      CLIENT_TYPE_INSTALLED_APP = 'installed'

      def __init__(self, client_type=None, client_id=None, client_secret=None,
                   auth_uri=_DEFAULT_AUTH_URI, token_uri=_DEFAULT_TOKEN_URI):
          self.client_type = client_type
          self.client_id = client_id
          self.client_secret = client_secret
          self.auth_uri = auth_uri
          self.token_uri = token_uri

      def Build(self):
          """Builds a client config dictionary used in the OAuth 2.0 flow."""
          if all((self.client_type, self.client_id, self.client_secret,
                  self.auth_uri, self.token_uri)):
              client_config = {
                  self.client_type: {
                      'client_id': self.client_id,
                      'client_secret': self.client_secret,
                      'auth_uri': self.auth_uri,
                      'token_uri': self.token_uri
                  }
              }
          else:
              raise ValueError('Required field is missing.')

          return client_config


client_config = ClientConfigBuilder(
    client_type=ClientConfigBuilder.CLIENT_TYPE_WEB, client_id=client_id,
    client_secret=client_secret)

flow = InstalledAppFlow.from_client_config(
     client_config.Build(), scopes=SCOPES)
# Note that from_client_config will not produce a flow with the
# redirect_uris (if any) set in the client_config. This must be set
# separately.
flow.redirect_uri = 'http://localhost:8080'

auth_url, _ = flow.authorization_url(prompt='consent')

print('Log into the Google Account you use to access your AdWords account '
      'and go to the following URL: \n%s\n' % auth_url)
print('After approving the token copy and paste the full URL.')
url = input('URL: ').strip()
code = parse.parse_qs(parse.urlparse(url).query)['code'][0]

try:
    flow.fetch_token(code=code)
except InvalidGrantError as ex:
    print('Authentication has failed: %s' % ex)
    sys.exit(1)

token = flow.credentials.token
refresh_token = flow.credentials.refresh_token

Credentials Check

In [ ]:
#@title
from googleapiclient import discovery
from apiclient.http import MediaFileUpload, HttpRequest
from google.oauth2.credentials import Credentials
credentials = None

def get_credentials(token, refresh_token, client_id, client_secret):
    global credentials
    if credentials == None or credentials.expired:
      credentials = Credentials(
        token=token,
        refresh_token=refresh_token,
        client_id=client_id,
        client_secret=client_secret,
        token_uri=ClientConfigBuilder._DEFAULT_TOKEN_URI,
        scopes=SCOPES)
    return credentials

# Build the discovery document URL.
discovery_url = f'https://displayvideo.googleapis.com/$discovery/rest?version=v2'

# Build the API service.
service = discovery.build(
    'displayvideo',
    'v2',
    discoveryServiceUrl=discovery_url,
    credentials=get_credentials(token, refresh_token, client_id, client_secret))

print(service.advertisers().list(partnerId=partnerId).execute())



Upload Script to DV360

In [ ]:
# @title

# Create a custom bidding algorithm object.
custom_bidding_algorithm_obj = {
    'advertiserId': advertiserId,
    'displayName': 'GAP-autogenerated-custom-bidding-script',
    'entityStatus': 'ENTITY_STATUS_ACTIVE',
    'customBiddingAlgorithmType': 'SCRIPT_BASED'
}

# Create the custom bidding algorithm.
response = service.customBiddingAlgorithms().create(
    body=custom_bidding_algorithm_obj
).execute()

# Display the new custom bidding algorithm.
print(f'The following Custom Bidding Algorithm was created: {response}')

# Retrieve a usable custom bidding script reference
# object.
algorithmId = response['customBiddingAlgorithmId']
custom_bidding_script_ref = service.customBiddingAlgorithms().uploadScript(
    customBiddingAlgorithmId=algorithmId,
    advertiserId=advertiserId
).execute()

# Display the new custom bidding script reference object.
print('The following custom bidding script reference object was retrieved:'
      f'{custom_bidding_script_ref}')

# Create a media upload object.
media = MediaFileUpload('/content/script.txt')

# Create upload request.
upload_request = service.media().upload(
    resourceName=custom_bidding_script_ref['resourceName'], media_body=media)

# Override response handler to expect null response.
upload_request.postproc = HttpRequest.null_postproc

# Upload script to resource location given in retrieved custom bidding
# script reference object.
upload_request.execute()

# Create a custom bidding script object.
script_obj = {
    'script': custom_bidding_script_ref
}

# Create the custom bidding script.
response = service.customBiddingAlgorithms().scripts().create(
    customBiddingAlgorithmId=algorithmId,
    advertiserId=advertiserId,
    body=script_obj).execute()

# Display the new custom bidding script object.
print(f'The following custom bidding script was created: {response}')

# Create the new bid strategy object.
bidding_strategy = {
    'maximizeSpendAutoBid': {
        'performanceGoalType':
            'BIDDING_STRATEGY_PERFORMANCE_GOAL_TYPE_CUSTOM_ALGO',
        'customBiddingAlgorithmId': algorithmId
    }
}

# Create a line item object assigning the new bid strategy.
line_item_obj = {'bidStrategy': bidding_strategy}

# Update the line item with a new bid strategy.
#response = service.advertisers().lineItems().patch(
#    advertiserId=advertiserId,
#    lineItemId=line-item-id,
#    updateMask='bidStrategy',
#    body=line_item_obj).execute()

# Display the line item's new bid strategy
#print(f'Line Item {response["name"]} is now using the following bid'
#     f' strategy: {response["bidStrategy"]}.')